Separately run referrals and claims queries.
This file imports the results of those queries, cleans and standardizes each, and merges them for analysis.

In addition, we import clinical decision files and use it to "override" decisions that were made by analyzing the ROI for specialty & cpt code combination.

Finally, several outputs are prepared for reporting purposes, including the calculation of a projected AA approval rate.

In [ ]:
## Import required packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pyodbc
import os

## Import Data

Import the most up-to-date spec_cpt_summary file. Note that clinical decisions should already have been added to this file using another script.

In [ ]:
cpts_w_claims_fin = pd.read_excel('../Data/AADictionary_Master.xlsx', sheet_name="Specialty Summary")

Create a list of relevent specialties

In [ ]:
cpts_w_claims_fin.head()

In [ ]:
cpts_w_claims_fin = cpts_w_claims_fin[['Specialty', 'final_decision', ]]

In [ ]:
list_o_specs = list(cpts_w_claims_fin['Specialty'].unique())

In [ ]:
## Function that gathers CPT codes that should be pended and stores them in a dictionary.
def create_dict_of_CPT_codes_v3(specialty_cpt, list_o_specs, PPL=1):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['Decision'] == 1:
            if row['is_PPL'] == PPL:
                spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict

In [ ]:
spec_dict_PPL = create_dict_of_CPT_codes_v3(cpts_w_claims_fin, list_o_specs, PPL=1)

## Generate Rule Spreadsheet that can be uploaded to CCT

CCT, the tool that houses the referrals auto-approval rules and processing, can auto-generate rules when provided a spreadsheet of a specific format.  This portion of the code reads in that file, and then completes it with the information required.

Read in file to get accurate headers and format

In [ ]:
rule_file = pd.read_excel('../Data/RMS_Rules.xlsx', sheet_name='RULE_AUTOAPPROVEPX')

Add codes to dictionary for specific specialties that appear in CCT data, but not the data warehouse

In [ ]:
spec_dict_PPL['HIV/AIDS SPECIALIST'] = ['99203', '99204', '99213', '99214']

In [ ]:
spec_dict_PPL['COMPREHENSIVE CARE PROGRAM - AMBULATORY CARE MGT, COMPREHENSIVE CARE PROGRAM - CHF, COMPREHENSIVE CARE PROGRAM - COMPREHEN CARE CENTER, COMPREHENSIVE CARE PROGRAM - ESRD/CKD, COMPREHENSIVE CARE PROGRAM - PALLIATIVE CARE, COMPREHENSIVE CARE PROGRAM - SOCIAL WORK, COMPREHENSIVE CARE PROGRAM -COPD, COMPREHENSIVE CARE PROGRAM -HOUSE CALLS'] = ['All']

Update list_o_specs to include specialties that appear in CCT data, but not in the data warehouse

In [ ]:
list_o_specs = list(spec_dict_PPL.keys())

In [ ]:
def create_rule_excel_file(rule_file, spec_dict, list_o_specs, PPL=True):
    rule_file['Auth_Status'] = 8
    rule_file['Auth_Status_Num'] = 1
    rule_file['RefType_Key'] = np.nan
    rule_file['RefType_Num'] = np.nan
    rule_file['ByPass_Adjudication'] = False
    rule_file['Region_List'] = 'All'
    rule_file['Rule_Priority'] = 2000
    rule_file['Rule_Category'] = 2
    rule_file['Eff_Date'] = pd.datetime(2019, 6, 15)
    rule_file['Term_Date'] = np.nan
    rule_file['Entity_List'] = 'All'
    rule_file['Site_List'] = 'All'
    rule_file['HMO_List'] = 'All'
    rule_file['Product_Type_List'] = 'All'
    rule_file['Dx_List'] = np.nan
    rule_file['Age_From'] = 18
    rule_file['Age_To'] = 199
    rule_file['Gender'] = 'A'
    rule_file['GoldStar'] = 'A'
    rule_file['POS_List'] = 'All'
    rule_file['RefTo_Provider_List'] = 'All'
    rule_file['RefFrom_Spc_List'] = 'All'
    rule_file['Facility_Spc_List'] = 'All'
    rule_file['Spc_Operator'] = 'OR'
    rule_file['Date_Added'] = pd.datetime.now()
    rule_file['Added_By'] = 'PIONEER'
    rule_file['Archive_Flag'] = False
    rule_file['RefFrom_Provider_List'] = 'ALL'
    rule_file['RefTo_Status_List'] = 'PPL'
    rule_key = list(rule_file['Rule_Key'])
    Px_list = list(rule_file['Px_List'])
    rule_code = list(rule_file['Rule_Code'])
    rule_desc = list(rule_file['Rule_Desc'])
    rule_lob = list(rule_file['LOB_List'])
    ref_to_spec = list(rule_file['RefTo_Spc_List'])
    index = 0
    for spec in list_o_specs:
        if len(spec_dict[spec]) == 0:
            continue
        else:
            rule_key[index] = index+1
            Px_list[index] = spec_dict[spec]
            rule_code[index] = (spec + '_PPL').replace(' ', '')
            rule_desc[index] = ('auto-approve ' + spec + ' referrals to PPL providers for specific codes.')
            if spec[-7:] == 'not_sen':
                ref_to_spec[index] = spec[:-8]
            elif spec[-6] == 'senior':
                ref_to_spec[index] = spec[:-7]
            else:
                ref_to_spec[index] = spec
            if spec[-6:]=='senior':
                rule_lob[index] = 'SENIOR'
            else:
                rule_lob[index] = 'All'
            index+=1
    rule_file['Rule_Key'] = rule_key
    rule_file['Px_List'] = Px_list
    rule_file['Rule_Desc'] = rule_desc
    rule_file['Rule_Code'] = rule_code
    rule_file['LOB_List'] = rule_lob
    rule_file['RefTo_Spc_List'] = ref_to_spec
    rule_file.drop(rule_file.index[index:], inplace=True)
    return rule_file

In [ ]:
new_rule_file = create_rule_excel_file(rule_file, spec_dict_PPL, list_o_specs)

Some specialties that are not in the data warehouse file need to be added to the "umbrella" specialty.  For example, CT - CAT SCANS needs to be added to the specialty field for RADIOLOGY.

In [ ]:
new_rule_file['RefTo_Spc_List'].replace({'RADIOLOGY':
                                         ['RADIOLOGY, CT - CAT SCANS, MAMMOGRAMS, MRI - MAGNETIC RESONANCE IMAGING', 'BONE DENSITY SCANS'],
                                        'CARDIOLOGY': ['CARDIOLOGY, CARDIAC TESTING']}, inplace=True)

In [ ]:
# Give the filename you wish to save the file to
rules_to_upload_filename = '../Data/Outputs/RMS_Rules_new.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(rules_to_upload_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    new_rule_file.to_excel(rules_to_upload_filename, index=False, )
else:
    print('WARNING: This file already exists!')